# Welcome to my LSTM Model! Hit run and watch the step-by-step training of my stock prediction Model

At the end you will get a LSTM_model file which can be plugged into a different script @/stockpredictionapp.ipynb which can be applied to all stocks and not just the 9 tested here!

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Define the tickers to fetch data for
tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NVDA', 'ADBE', 'INTC', 'CSCO', 'IBM']

# Fetch data for each ticker using Yahoo Finance API
data = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    data[ticker] = stock.history(period='5y')['Close']

# Combine data for all stocks into a single dataset
df = pd.concat(data.values(), axis=1, keys=data.keys())

df

,AAPL,GOOGL,MSFT,TSLA,NVDA,ADBE,INTC,CSCO,IBM
Date,,,,,,,,,
2018-04-23 00:00:00-04:00,39.381695,53.690498,89.988914,18.891333,55.430317,225.300003,44.250061,38.033005,109.461021
2018-04-24 00:00:00-04:00,38.833538,51.132000,87.884293,18.897333,54.766777,217.889999,44.553146,37.594662,109.235924
2018-04-25 00:00:00-04:00,39.002750,51.149502,87.119835,18.712667,53.642727,217.320007,44.492531,37.568863,109.521088
2018-04-26 00:00:00-04:00,39.138588,52.165501,88.960228,19.032000,55.762081,221.910004,45.938660,37.998619,110.106430
2018-04-27 00:00:00-04:00,38.685772,51.572498,90.432487,19.605333,56.036915,221.899994,45.661556,38.428371,109.926315
...,...,...,...,...,...,...,...,...,...
2023-04-17 00:00:00-04:00,165.229996,105.970001,288.799988,187.039993,270.019989,377.920013,32.139999,50.279999,127.820000
2023-04-18 00:00:00-04:00,166.470001,104.500000,288.369995,184.309998,276.670013,377.549988,31.830000,50.310001,127.779999
2023-04-19 00:00:00-04:00,167.630005,104.180000,288.450012,180.589996,279.309998,380.109985,31.150000,48.040001,126.320000


# This data will now be shaped so it can be inputted into the machine learning model!

Using MinMaxScaler we are able to normalize our dataframe into managble variables which can be inputted into the model

In [5]:
# scale the data
scaler = MinMaxScaler(feature_range=(0,10))
scaled_data = scaler.fit_transform(df)

# split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size, :]
test_data = scaled_data[train_size:, :]


# Creating Datasets

What is lookback?

    It is the value of N past time steps which the dataset will input into the neural network to predict the next value of the time-series.
    
This code will take that value and create datasets for testing and training the prediction values and the model.
    
    

In [6]:
# create input/output arrays for training and testing data
def create_dataset(data, lookback):
    X, Y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

lookback = 60
x_train, y_train = create_dataset(train_data, lookback)
x_test, y_test = create_dataset(test_data, lookback)


# Time to build the model!
1. creating the models
2. compliing the model
3. fitting the data into the model

In [ ]:

# Build Model
model = Sequential() 
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


# Heres what our model looks like so far!

In [ ]:
print(model.summary())

# Now that the model is created, we can start to make our predictions!


In [ ]:
# make predictions and evaluate performance
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error:', mae)

# pickle data
model.save("lstm_model_l1")
